In [1]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=0dcc769736fb3cb34a36aadb5a58650e343dc54e435d58fa6982379190d054c8
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

In [3]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

# Define the list of FAANG ticker symbols
tickers = ['AAPL', 'AMZN', 'NFLX', 'GOOGL']

# Set the start and end dates for the historical data
datetime_str = '06/17/23'

end_date = datetime.strptime(datetime_str, '%m/%d/%y')
#datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (end_date - timedelta(days=30*365)).strftime('%Y-%m-%d')

# Create an empty dataframe to store the stock data
combined_data_open = pd.DataFrame()
combined_data_close = pd.DataFrame()
combined_data_high = pd.DataFrame()
combined_data_low = pd.DataFrame()
combined_data_vol = pd.DataFrame()
combined_data_adj_cls = pd.DataFrame()
combined_data = pd.DataFrame()

# Download historical stock data for each ticker and combine into a single dataframe
for ticker in tickers:
    print(ticker)
    data = yf.download(ticker, start=start_date, end=end_date, interval = '1h')
    data.columns = ticker+"_"+data.columns  # Add a column to identify the ticker
    if combined_data_open.shape[0] ==0:
      combined_data = data
      combined_data_adj_cls = data[ticker+"_Adj Close"]
      combined_data_open = data[ticker+"_Open"]
      combined_data_close = data[ticker+"_Close"]
      combined_data_high = data[ticker+"_High"]
      combined_data_low = data[ticker+"_Low"]
      combined_data_vol = data[ticker+"_Volume"]
    else:
      combined_data = pd.merge(combined_data, data, left_index=True, right_index=True)
      combined_data_open = pd.merge(combined_data_open, data[ticker+"_Open"], left_index=True, right_index=True)
      combined_data_adj_cls = pd.merge(combined_data_adj_cls, data[ticker+"_Adj Close"], left_index=True, right_index=True)
      combined_data_close = pd.merge(combined_data_close, data[ticker+"_Close"], left_index=True, right_index=True)
      combined_data_high = pd.merge(combined_data_high, data[ticker+"_High"], left_index=True, right_index=True)
      combined_data_low = pd.merge(combined_data_low, data[ticker+"_Low"], left_index=True, right_index=True)
      combined_data_vol = pd.merge(combined_data_vol, data[ticker+"_Volume"], left_index=True, right_index=True)



AAPL
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAPL: 1h data not available for startTime=740894400 and endTime=1686974400. The requested range must be within the last 730 days.
AMZN
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AMZN: 1h data not available for startTime=863703000 and endTime=1686974400. The requested range must be within the last 730 days.
NFLX
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NFLX: 1h data not available for startTime=1022160600 and endTime=1686974400. The requested range must be within the last 730 days.
GOOGL
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GOOGL: 1h data not available for startTime=1092922200 and endTime=1686974400. The requested range must be within the last 730 days.


In [4]:
combined_data_adj_cls.shape

(0,)

In [5]:
combined_data_adj_cls

Series([], Name: GOOGL_Adj Close, dtype: float64)

In [6]:
combined_data_adj_cls.describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: GOOGL_Adj Close, dtype: float64

In [7]:
train_class = combined_data['GOOGL_Adj Close'].values
train_class

array([], dtype=float64)

In [8]:
y = 'GOOGL_Adj Close'
X = combined_data_adj_cls.loc[:, combined_data_adj_cls.columns != 'GOOGL_Adj Close'].columns.tolist()

AttributeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_data_adj_cls[X], train_class,
                                                    train_size=0.75, test_size=0.25)

In [ ]:
y_test

In [ ]:
plt.plot(X_train.index,y_train)
plt.plot(X_test.index,y_test)
plt.ylabel('GOOGLE Adj Close',fontsize=18)
plt.legend(['train','test'])
plt.show()

In [ ]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)


In [ ]:
print("TPOT cross-validation MSE")
print(tpot.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE:')
print(mean_squared_error(y_test, tpot.predict(X_test)))

In [ ]:
print('RMSE:')
print(np.sqrt(mean_squared_error(y_test, tpot.predict(X_test))))

In [ ]:
tpot.predict(combined_data_adj_cls[X])

In [ ]:
f = combined_data_adj_cls['GOOGL_Adj Close']
f.reset_index(inplace=True, drop=True)

In [ ]:
plt.plot(tpot.predict(combined_data_adj_cls[X]), label = 'Predictions')
plt.plot(f, label = 'Actual')
plt.legend()

Daily

In [ ]:
import yfinance as yf
import datetime
import pandas as pd

# Define the list of FAANG ticker symbols
tickers = ['AAPL', 'AMZN', 'NFLX', 'GOOGL']

# Set the start and end dates for the historical data
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=30*365)).strftime('%Y-%m-%d')

# Create an empty dataframe to store the stock data
combined_data_open = pd.DataFrame()
combined_data_close = pd.DataFrame()
combined_data_high = pd.DataFrame()
combined_data_low = pd.DataFrame()
combined_data_vol = pd.DataFrame()
combined_data_adj_cls = pd.DataFrame()
combined_data = pd.DataFrame()

# Download historical stock data for each ticker and combine into a single dataframe
for ticker in tickers:
    print(ticker)
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = ticker+"_"+data.columns  # Add a column to identify the ticker
    if combined_data_open.shape[0] ==0:
      combined_data = data
      combined_data_adj_cls = data[ticker+"_Adj Close"]
      combined_data_open = data[ticker+"_Open"]
      combined_data_close = data[ticker+"_Close"]
      combined_data_high = data[ticker+"_High"]
      combined_data_low = data[ticker+"_Low"]
      combined_data_vol = data[ticker+"_Volume"]
    else:
      combined_data = pd.merge(combined_data, data, left_index=True, right_index=True)
      combined_data_open = pd.merge(combined_data_open, data[ticker+"_Open"], left_index=True, right_index=True)
      combined_data_adj_cls = pd.merge(combined_data_adj_cls, data[ticker+"_Adj Close"], left_index=True, right_index=True)
      combined_data_close = pd.merge(combined_data_close, data[ticker+"_Close"], left_index=True, right_index=True)
      combined_data_high = pd.merge(combined_data_high, data[ticker+"_High"], left_index=True, right_index=True)
      combined_data_low = pd.merge(combined_data_low, data[ticker+"_Low"], left_index=True, right_index=True)
      combined_data_vol = pd.merge(combined_data_vol, data[ticker+"_Volume"], left_index=True, right_index=True)



In [ ]:
train_class = combined_data['GOOGL_Adj Close'].values
train_class

In [ ]:
y = 'GOOGL_Adj Close'
X = combined_data_adj_cls.loc[:, combined_data_adj_cls.columns != 'GOOGL_Adj Close'].columns.tolist()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_data_adj_cls[X], train_class,
                                                    train_size=0.75, test_size=0.25)

In [ ]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)


In [ ]:
print("TPOT cross-validation MSE")
print(tpot.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE:')
print(mean_squared_error(y_test, tpot.predict(X_test)))

In [ ]:
print('RMSE:')
print(np.sqrt(mean_squared_error(y_test, tpot.predict(X_test))))

In [ ]:
tpot.predict(combined_data_adj_cls[X])

In [ ]:
f = combined_data_adj_cls['GOOGL_Adj Close']
f.reset_index(inplace=True, drop=True)

In [ ]:
plt.plot(tpot.predict(combined_data_adj_cls[X]), label = 'Predictions')
plt.plot(f, label = 'Actual')
plt.legend()